# ТЕСТОВОЕ ЗАДАНИЕ НА ВАКАНСИЮ “Backend разработчик (Python, Junior)”

References:
- <a href="https://pandas.pydata.org/docs/index.html">pandas documentation</a>

In [95]:
import requests as rq
import pandas as pd
import numpy as np

In [2]:
path_to_data = "https://file.notion.so/f/s/0f8850ad-e46f-4f37-99ea-0e4e2a6af5b6/trial_task.json?id=2583a04b-4256-4c1f-939e-6eac0f749ceb&table=block&spaceId=41165294-a784-489a-a401-1a916d020564&expirationTimestamp=1689868800000&signature=-1D4yL8SjErQtHRmRLR8SDvsSk6n_DPQ5uefnI1oeLs&downloadName=trial_task.json"
response = rq.get(path_to_data)
response.status_code

200

In [3]:
data = response.json()
data

[{'order_id': 11973,
  'warehouse_name': 'Мордор',
  'highway_cost': -70,
  'products': [{'product': 'ломтик июльского неба',
    'price': 450,
    'quantity': 1},
   {'product': 'билет в Израиль', 'price': 1000, 'quantity': 3},
   {'product': 'статуэтка Ленина', 'price': 200, 'quantity': 3}]},
 {'order_id': 62239,
  'warehouse_name': 'хутор близ Диканьки',
  'highway_cost': -15,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 1}]},
 {'order_id': 85794,
  'warehouse_name': 'отель Лето',
  'highway_cost': -50,
  'products': [{'product': 'зеленая пластинка', 'price': 10, 'quantity': 2}]},
 {'order_id': 33684,
  'warehouse_name': 'Мордор',
  'highway_cost': -30,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 2},
   {'product': 'зеленая пластинка', 'price': 10, 'quantity': 1}]},
 {'order_id': 25824,
  'warehouse_name': 'отель Лето',
  'highway_cost': -75,
  'products': [{'product': 'автограф Стаса Барецкого',
    'price': 600,
    'quantit

In [4]:
df = pd.json_normalize(data, 'products', ['order_id', 'warehouse_name', 'highway_cost'])

reordered = ['order_id', 'product', 'price', 'quantity', 'warehouse_name', 'highway_cost']
df = df.reindex(columns=reordered)
df.rename(columns={'highway_cost': 'highway_cost_per_order'}, inplace=True)
df

,order_id,product,price,quantity,warehouse_name,highway_cost_per_order
0,11973,ломтик июльского неба,450,1,Мордор,-70
1,11973,билет в Израиль,1000,3,Мордор,-70
2,11973,статуэтка Ленина,200,3,Мордор,-70
3,62239,билет в Израиль,1000,1,хутор близ Диканьки,-15
4,85794,зеленая пластинка,10,2,отель Лето,-50
...,...,...,...,...,...,...
192,79293,автограф Стаса Барецкого,600,1,отель Лето,-75
193,79293,ломтик июльского неба,450,1,отель Лето,-75
194,2930,плюмбус,250,2,Мордор,-30
195,2930,плюмбус,250,1,Мордор,-30


## 1. Тариф стоимости доставки для каждого склада

In [5]:
grouped = df.groupby(['warehouse_name', 'order_id'])
tariffs = grouped.highway_cost_per_order.first() // grouped.quantity.sum()
tariffs = tariffs.groupby('warehouse_name').first()

print('Cтоимость доставки для каждого склада:')
print(pd.DataFrame(tariffs, columns=['tariff']))

Cтоимость доставки для каждого склада:
                    tariff
warehouse_name            
Мордор                 -10
гиперборея             -20
остров невезения        -5
отель Лето             -25
хутор близ Диканьки    -15


## 2. Суммарное количество , суммарный доход , суммарный расход и суммарная прибыль для каждого товара

In [6]:
summary = df.drop('highway_cost_per_order', axis=1)
summary.insert(4, 'expenses', summary.quantity * [tariffs[value] for value in summary.warehouse_name.values])
summary.insert(4, 'income', summary.price * summary.quantity)
summary.insert(6, 'profit', (summary.income + summary.expenses))

product_slice = summary[['product', 'quantity', 'income', 'expenses', 'profit']]
product_totals = product_slice.groupby('product', as_index=False).sum()
print(product_totals)

                    product  quantity  income  expenses  profit
0  автограф Стаса Барецкого        48   28800      -820   27980
1           билет в Израиль        58   58000     -1050   56950
2         зеленая пластинка        61     610      -920    -310
3     ломтик июльского неба        42   18900      -670   18230
4                   плюмбус        65   16250      -940   15310
5    подписка на suppi-блог        33    4950      -545    4405
6          статуэтка Ленина        68   13600      -935   12665


## 3. 'order_id' (id заказа), 'order_profit' (прибыль полученная с заказа) и средняя прибыль заказов

In [7]:
order_slice = summary[['order_id', 'profit']]
profit_per_order = pd.DataFrame(order_slice.groupby('order_id').sum())
profit_per_order = profit_per_order.rename(columns={'profit': 'order_profit'})
print('Прибыль с каждого заказа:')
print(profit_per_order)
print()
print('Средняя прибыль по всем заказам:', profit_per_order.order_profit.mean())

Прибыль с каждого заказа:
          order_profit
order_id              
124                705
1391               490
2091              1300
2108               200
2558               355
...                ...
98100             1590
98423             1170
99220             1075
99246             1755
99324             1590

[100 rows x 1 columns]

Средняя прибыль по всем заказам: 1352.3


## 4. Статистика по складам

- 'warehouse_name', 'product','quantity', 'profit', 'percent_profit_product_of_warehouse'

In [10]:
wh_slice = summary[['warehouse_name', 'product','quantity', 'profit']]
profit_per_wh = wh_slice[['warehouse_name', 'profit']].groupby('warehouse_name').sum()
print(profit_per_wh)

                     profit
warehouse_name             
Мордор                17050
гиперборея            38280
остров невезения      15610
отель Лето            27050
хутор близ Диканьки   37240


In [9]:
profit_per_product_across_wh = wh_slice.groupby(['warehouse_name' , 'product'], as_index=False).sum()

ppwh = profit_per_product_across_wh
ppwh['percent_profit_product_of_warehouse'] = ppwh.profit / ppwh.warehouse_name.apply(lambda x: profit_per_wh.loc[x].profit)  * 100
with pd.option_context("display.width", 120):
    print(ppwh)

         warehouse_name                   product  quantity  profit  percent_profit_product_of_warehouse
0                Мордор  автограф Стаса Барецкого         4    2360                            13.841642
1                Мордор           билет в Израиль         9    8910                            52.258065
2                Мордор         зеленая пластинка        11       0                             0.000000
3                Мордор     ломтик июльского неба         3    1320                             7.741935
4                Мордор                   плюмбус         6    1440                             8.445748
5                Мордор    подписка на suppi-блог         8    1120                             6.568915
6                Мордор          статуэтка Ленина        10    1900                            11.143695
7            гиперборея  автограф Стаса Барецкого        12    6960                            18.181818
8            гиперборея           билет в Израиль      

## 5. Накопленный процент

In [103]:
percent_sorted = ppwh.sort_values(['warehouse_name', 'percent_profit_product_of_warehouse'], ascending=[True, False]).reset_index(drop=True)

percent_grouped = percent_sorted.groupby('warehouse_name')
window = np.max([len(group) for name, group in percent_grouped])
accumulated = percent_grouped.percent_profit_product_of_warehouse.rolling(window, min_periods=1).sum().values

percent_sorted['accumulated_percent_profit_product_of_warehouse'] = accumulated
with pd.option_context("display.width", 160):
    print(percent_sorted)

         warehouse_name                   product  quantity  profit  percent_profit_product_of_warehouse  accumulated_percent_profit_product_of_warehouse
0                Мордор           билет в Израиль         9    8910                            52.258065                                        52.258065
1                Мордор  автограф Стаса Барецкого         4    2360                            13.841642                                        66.099707
2                Мордор          статуэтка Ленина        10    1900                            11.143695                                        77.243402
3                Мордор                   плюмбус         6    1440                             8.445748                                        85.689150
4                Мордор     ломтик июльского неба         3    1320                             7.741935                                        93.431085
5                Мордор    подписка на suppi-блог         8    1120         